<a href="https://colab.research.google.com/github/SandboxGTASA/0478_22_FebMarch_Pizza/blob/main/TENSORFLOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# Arquivo de treinamento e predicão do modelo de IA do programa IA Trader
# Autor: Matheus Lemos <matheuslemosf@gmail.com>

from os import environ
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, optimizers, Sequential, models
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# Uso de outro modelo
from sklearn.svm import SVR

class Model:
    """
    Classe que define o modelo de machine learning
    """

    def __init__(self,datasetName=None):
        """
        Assignment de variáveis
        """

        self.datasetName = datasetName

    def splitData(self, data, trainTestSplit):
        """
        Divide em dados de treinamento (train) e teste (test)

        Arguments:
            dataset {Pandas.DataFrame} -- DataFrame do pandas com os dados do dataset

        Returns:
            trainDataset {pandas.DataFrame} -- Dataset de treino
            trainDataset {pandas.DataFrame} -- Dataset de treino
            testDataset {pandas.DataFrame} -- Labels de treino
            testDataset {pandas.DataFrame} -- Labels de treino
        """

        # Carregamento dos dados
        trainDataset = data.sample(frac=trainTestSplit, random_state=1)
        testDataset = data.drop(trainDataset.index)
        trainLabels = trainDataset.pop('fechamento')
        testLabels = testDataset.pop('fechamento')

        return trainDataset, testDataset, trainLabels, testLabels

    def plotHistory(self, history):
        """
        Gera gráfico do histórico de treino (fit)

        Arguments:
            history {} -- Histórico de treino do keras.Sequential
        """

        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch

        plt.figure()
        plt.xlabel('Época')
        plt.ylabel('Média do erro absoluto')
        plt.plot(hist['epoch'], hist['mae'], label='Erro de treino')
        plt.plot(hist['epoch'], hist['val_mae'], label='Erro de validacão')
        plt.legend()
        
        plt.figure()
        plt.xlabel('Época')
        plt.ylabel('Média do erro absoluto')
        plt.plot(hist['epoch'], hist['mse'], label='Erro de treino')
        plt.plot(hist['epoch'], hist['val_mse'], label='Erro de validacão')
        plt.legend()
        plt.show()

    def load(self, dayDict):
        """
        Carrega o dataset a partir de arquivo CSV

        Arguments:
            dayDict {dict} -- Dicionário com os dias válidos de predicão, para fins de normalizacão
    
        Returns:
            df {pandas.DataFrame} -- DataFrame do pandas com os dados carregados e filtrados
        """

        # Busca dados no intervalo de tempo passado e remove colunas não necessárias
        precision = 4
        df = pd.read_csv(self.datasetName)
        df['fechamento'] = df['fechamento'].round(decimals=precision)
        
        # Faz o mapeamento de datas para valores

        df['dia'] = df['dia'].apply(lambda x: dayDict[str(x)])

        return df

    def normalize(self, data):
        """ Normaliza os dados por coluna

        Arguments:
            data {pandas.DataFrame} -- DataFrame do pandas com os dados brutos

        Returns:
            data {pandas.DataFrame} -- DataFrame do pandas com os dados normalizados
        """

        data['dia'] = (data['dia'] - data['dia'].mean()) / (data['dia'].max() - data['dia'].min())
        data['hora'] = (data['hora'] - data['hora'].mean()) / (data['hora'].max() - data['hora'].min())
        return data

    def write(self, fileName, data):
        """ Escreve o DataFrame em um arquivo CSV

        Arguments:
            fileName {str} -- Caminho do arquivo no sistema de arquivos onde os dados serão escritos
            data {pandas.DataFrame} -- DataFrame do pandas com os dados a serem escritos
        """
        data.to_csv(fileName, header=True, index=False)


    def fit(self, trainDataset, trainLabels, parameters):
        """
        Faz o treinamento do modelo e escreve os pesos em arquivo
        
        Arguments:
            trainDataset {pandas.DataFrame} -- Dataset de treino
            trainLabel {pandas.DataFrame} -- Labels de treino
            parameters {dict} -- Parâmetros de treinamento
        """

        # Variáveis
        epochs = int(parameters['epochs'])
       
        # Definicão do modelo
        model = Sequential([
            layers.Dense(24, kernel_initializer='normal', input_shape=[len(trainDataset.keys())]),
            layers.Dropout(0.5),
            layers.Dense(24, kernel_initializer='normal', activation='relu'),
            layers.Dense(1, kernel_initializer='normal')
        ])
        
        optimizer = optimizers.RMSprop(0.001)
        model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
        model.summary()

        # Treino do modelo
        history = model.fit(trainDataset, trainLabels, epochs=epochs, validation_split=0.2)

        # Salva o modelo em um arquivo
        model.save(parameters['modelfile'])
        
        # ESSA PARTE PODE SER DESCOMENTADA CASO DESEJE VER O GRÁFICO DE TREINAMENTO
        # Imprime o resultado em um gráfico
        #self.plotHistory(history)

    def predict(self, testDataset, testLabels, parameters):
        """
        Faz a predicão a partir dos dados de teste e gera gráfico 
        
        Arguments:
            testDataset {pandas.DataFrame} -- Dataset de teste 
            testLabel {pandas.DataFrame} -- Labels de teste
            parameters {dict} -- Dicionário com parâmetros de configuracão
        """

        # Carrega o modelo
        model = models.load_model(parameters['modelfile'])

        # Faz a predicão
        precision = 4
        predictions = model.predict(testDataset);

        # Avalia os resultados e imprime em tela
        predictedLabels = predictions.flatten()
        r2 = r2_score(testLabels, predictedLabels)
        mse = mean_squared_error(testLabels, predictedLabels)
        print("\n===== AVALIACÕES =====")
        print(f"R2-Score: {r2}")
        print(f"MSE: {mse}\n")
        
        # Converte para dataframe
        predictions = np.array([float(val) for sublist in predictions for val in sublist])
        predictions = [round(elem, precision) for elem in predictions]
        
        horas = np.array(testDataset['hora'].tolist())
        dias = np.array(testDataset['dia'].tolist())
        arr = np.array((predictions, dias, horas)).T

        df = pd.DataFrame(arr, columns=['fechamento', 'dia', 'hora'])   
        return df

    def validate(self, dia, parameters, dayDict):
        """ Cria previsões de valores do ativo para o dia passado como parâmetro

        Arguments:
            dia {int} -- Dia para que se deseja realizar as previsões
            parameters {dict} -- Dicionário de parâmetros do arquivo de configuracão
            dayDict {dict} -- Dicionário com todos os dias válidos para fins de validacão
        """

        # Carrega o modelo
        model = models.load_model(parameters['modelfile'])
    
        # Gera um DataFrame com os minutos de abertura da bolsa
        initMinute = 0
        endMinute = 59
        initHour = 900
        endHour = 1700
        i = initHour
        j = initMinute
        horas = []
        while i <= endHour:
            j = initMinute
            while j <= endMinute:
                horas.append(i + j)
                j += 1
            i += 100
            
        del horas[0:5]
        lenHoras = len(horas)
        dias = [int(dia)] * lenHoras
        
        horas = np.array(horas)
        dias = np.array(dias)
        arr = np.array((horas, dias)).T
        df = pd.DataFrame(arr, columns=['hora', 'dia'])
        print('dayDict')
        print(dayDict)
        print('df')
        print('Transforma o dia baseado no dicionário de dias ')
        # Transforma o dia baseado no dicionário de dias 
        df['dia'] = df['dia'].apply(lambda x: dayDict[str(x)])
        
        normalized  = df.copy()
        print('Carrega os dados de treinamento anteriores')
        # Carrega os dados de treinamento anteriores
        trainData = pd.read_csv('traindata.csv')
        
        # Normaliza os dados
        normalized['dia'] = (normalized['dia'] - trainData['dia'].mean()) / (trainData['dia'].max() - trainData['dia'].min())
        normalized['hora'] = (normalized['hora'] - trainData['hora'].mean()) / (trainData['hora'].max() - trainData['hora'].min())

        # Realiza a predicão
        prediction = model.predict(normalized).flatten()

        df['previsao'] = prediction
        print(df)

        # Escreve os dados
        predictionsFile = f'predictions/previsoes-{dia}.csv'
        df.to_csv(predictionsFile, header=True, index=False)

In [9]:
# Utilitário de interacão com arquivos
# Autor: Matheus Lemos <matheuslemosf@gmail.com>

import json

class FileDriver():
    """ Classe que interage com arquivos
    """

    def readJson(self, fileName):
        """ Método que faz a leitura de arquivos JSON e armazena em um dicionário

        Arguments:
            fileName {str} -- Caminho do arquivo no sistema de arquivos

        Returns:
            data {dict} -- Dicionário com os dados lidos do arquivo JSON
        """

        with open(fileName, 'r') as f:
            rawData = f.read()

        data = json.loads(rawData)
        return data
        
# Utilitário para formatar os dados de dia
# Autor: Matheus Lemos <matheuslemosf@gmail.com>

import datetime

class DateFormat():
    """ Classe para lidar com formatacão de datas
    """

    def genDayDict(self):
        """ Gera um dicionário dos dias 
        """

        # Variáveis
        year = 2020
        months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        dayDict = {}
        k = 1
        
        # Gera o dicionário
        for i in range(0, len(months)):
            j = 1
            for j in range(1, months[i] + 1):
                weekday = datetime.datetime(year, i + 1, j).weekday()
                key = f"{j}{i+1:02}"
                if weekday not in [5,6]:
                    dayDict[key] = k
                    k += 1
        
        return dayDict
        


In [43]:
# Arquivo da interface de texto
# Autor: Matheus Lemos <matheuslemosf@gmail.com>

# from model import Model
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from utils.fileDriver import FileDriver
# from utils.dateFormat import DateFormat

class Interface:
    """
    Classe responsável pela interface e validacão de input
    Arguments:
        model {Model} -- Classe do modelo do tensorflow
    """

    def __init__(self):
        """
        Assignment de variáveis
        """
        self.parameters = {}
        self.fileDriver = FileDriver()
        self.dateFormat = DateFormat()
        self.dias = self.dateFormat.genDayDict()

    def handleFit(self):
        """ Método responsável pelo treinamento e exibicão dos gráficos de treinamento do modelo
        """

        # Parâmetros
        parameterFileName = 'modelConfig.json'
        
        # Validacão e execucão
        # try:
        # Leitura do arquivo json de configuracões 
        config = self.fileDriver.readJson(parameterFileName)
        dias = self.dias 
        
        # Leitura dos dados CSV para datasets e labels
        csvFileName = config['dataset']['filename']
        model = Model(csvFileName)
        data = model.load(dias)
        
        # Escreve os dados para uso posterior
        model.write('traindata.csv', data)

        # Normaliza os dados
        normalizedData = model.normalize(data)
        trainDataset, testDataset, trainLabels, testLabels = model.splitData(normalizedData, float(config['dataset']['split']))
        
        # Criacão e treinamento do modelo
        model.fit(trainDataset, trainLabels, config['model'])

        # Verificacão do modelo com dados de teste
        predictions = model.predict(testDataset, testLabels, config['model'])
        
        # Gráfico predicoes x dia
        trainData = np.array((trainLabels, trainDataset['dia'], trainDataset['hora'])).T
        trainData = pd.DataFrame(trainData, columns=['fechamento', 'dia', 'hora'])

        sns.regplot(x='dia', y='fechamento', data=predictions)
        sns.regplot(x="dia", y='fechamento', data=trainData)
        plt.show()

        # Gráfico predicoes x hora
        sns.regplot(x='hora', y='fechamento', data=predictions)
        sns.regplot(x="hora", y='fechamento', data=trainData)
        plt.show()

        # except FileNotFoundError:
        #     print("Arquivo não foi encontrado")
        
    def handleValidate(self):
        """ Método responsável pela previsão de novos dados a partir de dia requisitado
        pelo usuário
        """

        # Parâmetros
        dia = input('Dia para que se deseja realizar predicão: ')
        parameterFileName = 'modelConfig.json'

        # try:
        # Leitura do arquivo json de configuracão
        config = self.fileDriver.readJson(parameterFileName)
        dias = self.dias 
    
        # Faz a predicão para esse único dado
        model = Model()
        model.validate(dia, config['model'], dias)

        # except FileNotFoundError:
        #     print("Arquivo não foi encontrado")
        # except KeyError:
        #     print("Valor inválido - Dia informado não pode ser fim de semana")

    def getUserInput(self):
        """
        Interface com o usuário
        """
        # Variáveis
        flag = False

        inputString = """Opcões:
        0 - Treinar modelo
        1 - Fazer predicões a partir de dados reais
Sua opcão: """

        while flag is False:
            option = input(inputString)

            if option == '0':
                flag = True
                self.handleFit()
            elif option == '1':
                flag = True
                self.handleValidate()
            else:
                print("Opcão inválida")

In [68]:
# Arquivo principal do programa IA Trader
# Autor: Matheus Lemos <matheuslemosf@gmail.com>

# from model import Model
# from interface import Interface

def main():
    """
    Funcão principal do projeto
    """
    interface = Interface()
    interface.getUserInput()

main()

Opcões:
        0 - Treinar modelo
        1 - Fazer predicões a partir de dados reais
Sua opcão: 1
Dia para que se deseja realizar predicão: 24
dayDict
{'101': 1, '201': 2, '301': 3, '601': 4, '701': 5, '801': 6, '901': 7, '1001': 8, '1301': 9, '1401': 10, '1501': 11, '1601': 12, '1701': 13, '2001': 14, '2101': 15, '2201': 16, '2301': 17, '2401': 18, '2701': 19, '2801': 20, '2901': 21, '3001': 22, '3101': 23, '302': 24, '402': 25, '502': 26, '602': 27, '702': 28, '1002': 29, '1102': 30, '1202': 31, '1302': 32, '1402': 33, '1702': 34, '1802': 35, '1902': 36, '2002': 37, '2102': 38, '2402': 39, '2502': 40, '2602': 41, '2702': 42, '2802': 43, '203': 44, '303': 45, '403': 46, '503': 47, '603': 48, '903': 49, '1003': 50, '1103': 51, '1203': 52, '1303': 53, '1603': 54, '1703': 55, '1803': 56, '1903': 57, '2003': 58, '2303': 59, '2403': 60, '2503': 61, '2603': 62, '2703': 63, '3003': 64, '3103': 65, '104': 66, '204': 67, '304': 68, '604': 69, '704': 70, '804': 71, '904': 72, '1004': 73, '130

KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/JOBS/TENSORFLOW/')